# 机器学习模型进行multi-lable分类（一级标签），步骤：

1. 读取数据，并将数据切分至train data、cross validation data、test data
2. 文本向量化：onehot与tfidf
3. 模型训练：多标签模型knn、svm
4. 交叉验证cross validation，选择优参数
5. 用test data 输出不同模型的准确度，给出展示。

## 1. 读取数据，并将数据切分至train data、cross validation data、test data

In [68]:
# 首先读取文本
import pandas as pd
MaoTi_all = pd.read_csv('Data\MaoTi_all_1ji_lable.0205') # 读取锚题
ZhenTi_all = pd.read_csv('Data\ZhenTi_all_1ji_lable.0205') # 读取真题，获取x
print(MaoTi_all.head())
print(ZhenTi_all.head())

# 用于获取 lable
MaoTi_source = pd.read_excel('Data\MaoTi_source.xlsx')
ZhenTi_source = pd.read_excel('Data\\2017年高考数学试卷标注统计.xlsx')

                                     id  \
0  2eff3829-23bd-401f-a283-5a73d1695118   
1  7282cc20-3d24-41ab-a599-5e08f0d0940b   
2  a8c3c018-e006-42d3-8cd1-0f531e4a8846   
3  4b284b4f-e58c-4c25-b47a-23003efc157f   
4  43421dd5-6538-498a-8fde-4310f7c0c537   

                                            pure_fea 1ji_lable  
0  数列 ENUM_SET SEQ_ELEMENT EQUAL NUM VARa VARn NU...         1  
1  数列 ENUM_SET 是以 NUM 首项 NUM 公差 等差数列 数列 ENUM_SET ...       1,2  
2  ENUM_SET 正项 等比数列 SEQ_SUM EQUAL LOG SEQ_ELEMENT...         1  
3  数据 NUM NUM NUM NUM NUM NUM 标准差 TARGET NUM NUM ...       2,4  
4  解 VARx 不等式 FRAC INEQUAL NUM nan 解 原 不等式 VARx N...         0  
                                     id  \
0  19ceb8ad-e997-47c3-a77a-f28468f6631d   
1  201c1bda-f139-4c3d-80cc-13e28d519985   
2  d7b2f640-f5c2-4c7a-a684-48e2f44ddff0   
3  b3f6c713-1851-45e9-b6ae-960b47e381fe   
4  07ec1e1f-ba0f-423b-acc0-089d9eaa9440   

                                            pure_fea   1ji_lable  
0  集合 VARA ENUM_SET VARA 

In [11]:
# 1. 数据集全部由锚题构成（因为锚题覆盖广 质量高700个）
train_x = MaoTi_all['pure_fea']

# 获取一级标签
train_y=[]
for i in range(0,len(MaoTi_source)): # 注意：这个切片参数，锚题从0行开始遍历，真题从1行开始遍历
    onehot_y = []
    if MaoTi_source.iloc[i,1] == 1 or MaoTi_source.iloc[i,2] == 1 or MaoTi_source.iloc[i,3] == 1: # A
        onehot_y.append('A')
    if MaoTi_source.iloc[i,4] == 1 or MaoTi_source.iloc[i,5] == 1 or MaoTi_source.iloc[i,6] == 1 or MaoTi_source.iloc[i,7] == 1: # B
        onehot_y.append('B')
    if MaoTi_source.iloc[i,8] == 1 or MaoTi_source.iloc[i,9] == 1 or MaoTi_source.iloc[i,10] == 1: # C
        onehot_y.append('C')
    if MaoTi_source.iloc[i,11] == 1 or MaoTi_source.iloc[i,12] == 1 or MaoTi_source.iloc[i,13] == 1 or MaoTi_source.iloc[i,14] == 1: # D
        onehot_y.append('D')
    if MaoTi_source.iloc[i,15] == 1 or MaoTi_source.iloc[i,16] == 1 or MaoTi_source.iloc[i,17] == 1: # E
        onehot_y.append('E')
    if MaoTi_source.iloc[i,18] == 1 or MaoTi_source.iloc[i,19] == 1 or MaoTi_source.iloc[i,20] == 1 or MaoTi_source.iloc[i,21] == 1: # F
        onehot_y.append('F')
    
    train_y.append(onehot_y)
print(len(train_y)) # [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0
print(train_y[:5])


700
[['B'], ['B', 'C'], ['B'], ['C', 'E'], ['A']]


In [12]:
# 2. 真题集（各329个）
ZhenTi_x = ZhenTi_all['pure_fea']
ZhenTi_y = []

for i in range(1,len(ZhenTi_source)): # 注意：这个切片参数，锚题从0行开始遍历，真题从1行开始遍历
    onehot_y = []
    if ZhenTi_source.iloc[i,1] == 1 or ZhenTi_source.iloc[i,2] == 1 or ZhenTi_source.iloc[i,3] == 1: # A
        onehot_y.append('A')
    if ZhenTi_source.iloc[i,4] == 1 or ZhenTi_source.iloc[i,5] == 1 or ZhenTi_source.iloc[i,6] == 1 or ZhenTi_source.iloc[i,7] == 1: # B
        onehot_y.append('B')
    if ZhenTi_source.iloc[i,8] == 1 or ZhenTi_source.iloc[i,9] == 1 or ZhenTi_source.iloc[i,10] == 1: # C
        onehot_y.append('C')
    if ZhenTi_source.iloc[i,11] == 1 or ZhenTi_source.iloc[i,12] == 1 or ZhenTi_source.iloc[i,13] == 1 or ZhenTi_source.iloc[i,14] == 1: # D
        onehot_y.append('D')
    if ZhenTi_source.iloc[i,15] == 1 or ZhenTi_source.iloc[i,16] == 1 or ZhenTi_source.iloc[i,17] == 1: # E
        onehot_y.append('E')
    if ZhenTi_source.iloc[i,18] == 1 or ZhenTi_source.iloc[i,19] == 1 or ZhenTi_source.iloc[i,20] == 1 or ZhenTi_source.iloc[i,21] == 1: # F
        onehot_y.append('F')
    
    ZhenTi_y.append(onehot_y)
print(len(ZhenTi_y)) # [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0
print(ZhenTi_y[:5])



329
[['C'], ['C'], ['B', 'C'], ['C', 'D'], ['A', 'B']]


In [69]:
# 划分数据集：把真题分割为 交叉验证集 与测试集（各164个）

from sklearn.model_selection import train_test_split

crossVali_x, test_x, crossVali_y, test_y = train_test_split(ZhenTi_x, ZhenTi_y, test_size=0.5)

print(train_x[:5],train_y[:5],len(train_y))
print('------------------------------------------------------------------------------------')
print(crossVali_x[:5],crossVali_y[:5],len(crossVali_y))
print('------------------------------------------------------------------------------------')
print(test_x[:5],test_y[:5],len(test_y))

0    数列 ENUM_SET SEQ_ELEMENT EQUAL NUM VARa VARn NU...
1    数列 ENUM_SET 是以 NUM 首项 NUM 公差 等差数列 数列 ENUM_SET ...
2    ENUM_SET 正项 等比数列 SEQ_SUM EQUAL LOG SEQ_ELEMENT...
3    数据 NUM NUM NUM NUM NUM NUM 标准差 TARGET NUM NUM ...
4    解 VARx 不等式 FRAC INEQUAL NUM nan 解 原 不等式 VARx N...
Name: pure_fea, dtype: object [['B'], ['B', 'C'], ['B'], ['C', 'E'], ['A']] 700
------------------------------------------------------------------------------------
320    TRIANGLE LINE EQUAL LINE EQUAL NUM LINE EQUAL ...
23     VARx VARy PIECEWISE VARx NUM VARy 最大值 TARGET N...
109    函数 FUN_VALUE_VAR EQUAL LOG VARx PLUS VAR_QUA P...
98     正方体 4PRISM VARE 棱 ARC 中点 TARGET LINE PERP LINE...
28     平面 直角 坐标系 COOTDINATE_SYSTEM 角 ALPHA 角 BETA 均以 ...
Name: pure_fea, dtype: object [['C', 'D'], ['C', 'D'], ['A', 'B', 'C'], ['B', 'C', 'D'], ['A', 'B']] 164
------------------------------------------------------------------------------------
160    设有 下面 四个 命题 SEQ_ELEMENT 复数 VARz FRAC BELONG VA...
75     椭圆 VARC OVAL VARa INE

In [70]:
# 把标签 01化！
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes = ['A','B','C','D','E','F'])
train_y_lable = mlb.fit_transform(train_y)
crossVali_y_lable = mlb.fit_transform(crossVali_y)
test_y_lable = mlb.fit_transform(test_y)
print(mlb.classes_)

print(train_y_lable[:3],crossVali_y_lable[:3],test_y_lable[:3])

['A' 'B' 'C' 'D' 'E' 'F']
[[0 1 0 0 0 0]
 [0 1 1 0 0 0]
 [0 1 0 0 0 0]] [[0 0 1 1 0 0]
 [0 0 1 1 0 0]
 [1 1 1 0 0 0]] [[0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]]


## 2. 文本向量化：onehot与tfidf

In [71]:
# 构建向量训练集（所有的特征文本x之和）

data_samples = MaoTi_all['pure_fea'].append(ZhenTi_all['pure_fea'], ignore_index=True)
print(data_samples)

0       数列 ENUM_SET SEQ_ELEMENT EQUAL NUM VARa VARn NU...
1       数列 ENUM_SET 是以 NUM 首项 NUM 公差 等差数列 数列 ENUM_SET ...
2       ENUM_SET 正项 等比数列 SEQ_SUM EQUAL LOG SEQ_ELEMENT...
3       数据 NUM NUM NUM NUM NUM NUM 标准差 TARGET NUM NUM ...
4       解 VARx 不等式 FRAC INEQUAL NUM nan 解 原 不等式 VARx N...
5       函数 FUN_VALUE_VAR EQUAL ABS PLUS ABS FUN_VALUE_...
6       等比数列 ENUM_SET SEQ_ELEMENT PLUS SEQ_ELEMENT PLU...
7       不等式 组 PIECEWISE 平面 区域内 整点 横坐标 和 纵坐标 都是 整数 点 个数...
8       IMG 四 棱锥 5HEDRON VARE ARC 中点 VARF ARC 中点 LINE ...
9       实数 VARx VARy PIECEWISE NUM VARx NUM VARy 取值 范围...
10      四个 几何体 三视图 这四个 几何体 依次 分 别是 TARGET IMG 三 棱台 三棱柱...
11      IMG 如图所示 三棱锥 4HEDRON LINE PERP 平面 PLANE LINE P...
12      IMG 4PRISM 正方体 下面 结论 正确 TARGET LINE PARAL 平面 P...
13      不等式 FRAC_SYMBOL VAR_POW NUM NUM NUM VARx MINUS...
14      直线 LINE_EQUA 直线 LINE_EQUA 平行 这两条 平行线 之间 距离 nan...
15      ENUM_SET 等比数列 SEQ_ELEMENT PLUS SEQ_ELEMENT EQU...
16      点 POINT_3DIM COOTDINATE_SYSTEM 平面 对称点 坐标 TARGE...
17      某人 楼房 

In [16]:
# 构建 词频tf 向量
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(data_samples) # 训练文本词典

train_x_tf_vct = tf_vectorizer.transform(train_x)
crossVali_x_tf_vct = tf_vectorizer.transform(crossVali_x)
test_x_tf_vct = tf_vectorizer.transform(test_x)

print(crossVali_x_tf_vct)

  (0, 61)	1
  (0, 66)	2
  (0, 68)	6
  (0, 74)	1
  (0, 93)	1
  (0, 94)	19
  (0, 95)	18
  (0, 101)	6
  (0, 102)	1
  (0, 103)	1
  (0, 105)	1
  (0, 114)	2
  (0, 116)	1
  (0, 119)	1
  (0, 123)	3
  (0, 127)	1
  (0, 134)	8
  (0, 137)	1
  (0, 141)	38
  (0, 142)	11
  (0, 143)	6
  (0, 147)	1
  (0, 160)	10
  (0, 163)	2
  (0, 165)	2
  :	:
  (163, 160)	7
  (163, 180)	6
  (163, 201)	1
  (163, 209)	2
  (163, 218)	3
  (163, 225)	1
  (163, 227)	1
  (163, 231)	5
  (163, 232)	1
  (163, 233)	1
  (163, 340)	1
  (163, 529)	1
  (163, 859)	1
  (163, 926)	1
  (163, 975)	1
  (163, 1337)	1
  (163, 1355)	2
  (163, 1427)	1
  (163, 1492)	1
  (163, 1555)	1
  (163, 1941)	1
  (163, 2316)	1
  (163, 2567)	3
  (163, 3257)	1
  (163, 3530)	1


In [185]:
# 构建 tfidf 向量
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(data_samples) # 训练文本词典

train_x_tfidf_vct = tfidf_vectorizer.transform(train_x)
crossVali_x_tfidf_vct = tfidf_vectorizer.transform(crossVali_x)
test_x_tfidf_vct = tfidf_vectorizer.transform(test_x)

print(crossVali_x_tfidf_vct)

  (0, 3640)	0.04369266113202153
  (0, 3479)	0.05831451861070061
  (0, 3384)	0.037312510458715566
  (0, 3019)	0.05540263789462346
  (0, 2972)	0.023888027768830315
  (0, 2968)	0.06230599994840418
  (0, 2856)	0.048881517258518935
  (0, 2563)	0.09683129768628385
  (0, 2333)	0.054782209915695455
  (0, 1941)	0.028063920961139504
  (0, 1840)	0.06868615062171016
  (0, 1728)	0.02170665808665406
  (0, 1510)	0.022325401494372208
  (0, 1141)	0.039303073441450107
  (0, 1096)	0.06230599994840418
  (0, 883)	0.042658113490569444
  (0, 859)	0.02756130642663402
  (0, 597)	0.05165955491542094
  (0, 590)	0.059183344948129664
  (0, 531)	0.08571456970110505
  (0, 342)	0.04347771282948053
  (0, 292)	0.04725841988298674
  (0, 222)	0.04459772968865918
  (0, 220)	0.030733188425426508
  (0, 213)	0.07691229963623665
  :	:
  (163, 751)	0.04507689421621769
  (163, 597)	0.049586922352193086
  (163, 529)	0.026502771448519504
  (163, 342)	0.12520003164890794
  (163, 340)	0.03653433502351063
  (163, 211)	0.250359511730

In [73]:
# 使用word2vec词向量特征
# 首先 读取模型（模型在95服务器上训练）

from gensim.models import Word2Vec
# 分词
def getSentences(path):
    '''
    获得用于词向量训练的句子
    :param path: 文件存储的路径
    :return: list[[单词],[单词].....]
    '''
    sents = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            if line != "":
                line_shuzu = line.split(" ")  # 按照空格拆分
                sents.append(line_shuzu)
    return sents

# 若第一次 运行 构建模型
# path = "Data\embfea1.0205"
# sentences = getSentences(path)
# model = Word2Vec(sentences, min_count=5, size=100, workers=2, window=5, iter=10)
# model.save("Data\emb_my_100.bin")  # 保存模型

# 后面载入模型
model = Word2Vec.load("Data\emb_my_100.bin")

print("函数：",model["函数"])  # 打印函数的词向量
print("NUM：",model["NUM"])  # 打印NUM的词向量


函数： [-1.7640394e+00  4.2826529e+00 -2.0158744e+00  4.3612938e+00
 -1.4179142e-01 -3.9970086e+00  5.0811305e+00  3.0308080e+00
  1.7861323e+00  1.8621773e+00  2.9964063e+00 -1.0679901e-02
  7.3306009e-02 -3.9404211e+00 -1.1000518e+00 -4.8098516e-01
 -1.1832634e-02 -2.4092910e+00  6.8852174e-01 -1.8851600e+00
  1.3572416e+00 -2.2222576e-01  3.3028390e+00  2.7103631e+00
 -6.2310886e-01 -2.0284886e+00  1.2591255e+00 -9.4247746e-01
 -2.0697088e+00 -2.4347372e+00  3.5791700e+00  4.8177678e-02
  4.4821227e-01 -7.5631404e-01  2.4531932e+00 -1.6476452e+00
 -2.1549761e+00 -2.4570816e+00  1.3004451e-01  3.8428608e-01
 -2.3210716e-02  1.8017774e+00 -2.4330514e-03 -1.0911762e+00
  8.7242931e-01  2.4945409e+00 -1.9489281e+00  2.6281452e-01
 -7.4507135e-01 -2.4338728e-01  4.3852153e+00 -3.2129185e+00
 -5.4083955e-01 -6.1659431e-01  3.9333200e+00  1.9449939e+00
  3.6641340e+00 -4.0597546e-01  1.2550095e+00 -1.4503902e+00
  1.0216436e+00 -5.3371626e-01 -3.4621243e+00 -2.5685060e-01
  2.7214766e+00  1.6

C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [74]:
print(model)

print (model.most_similar('函数'))
print(model.most_similar('NUM'))

Word2Vec(vocab=20603, size=100, alpha=0.025)


C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('FUN_VALUE_VAR', 0.6813198328018188), ('偶函数', 0.5755622982978821), ('奇函数', 0.558364748954773), ('原函数', 0.5321136713027954), ('LOG', 0.49604547023773193), ('FUN_VALUE', 0.4925633370876312), ('区间', 0.4851030111312866), ('导函数', 0.48485732078552246), ('一次函数', 0.4819476008415222), ('减函数', 0.4701527953147888)]
[('NUM\n', 0.7388532161712646), ('PLUS', 0.7082787752151489), ('MINUS', 0.685769259929657), ('VAR_QUA', 0.6710989475250244), ('EQUAL', 0.6578972339630127), ('POW', 0.6520572304725647), ('VARx', 0.6335667371749878), ('FRAC', 0.6303172707557678), ('VARa', 0.6237950325012207), ('INEQUAL', 0.6174031496047974)]


C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


In [196]:
# 用简单的方法构建句子特征。 词向量相加：
import numpy as np
from sklearn import preprocessing

def transformX2vec(test_xx): # 输入x给出 vec形式的数据
    test_xx_vec =[]
    for sentence in test_xx:
        sentence_vec = np.zeros(100) # 向量维度100
        for word in sentence :
            if word in model:
                word_vec = model[word]
                sentence_vec += word_vec
        test_xx_vec.append(sentence_vec)
    print ('转换完毕，样本个数：',len(test_xx_vec))
#     test_xxx_vec = preprocessing.scale(test_xx_vec)# 标准化向量
    return test_xx_vec

# 把三个样本进行向量化
train_x_vct = transformX2vec(train_x)
crossVali_x_vct = transformX2vec(crossVali_x)
test_x_vct = transformX2vec(test_x)

print (train_x_vct[0])
print (crossVali_x_vct[0])
print (test_x_vct[0])

C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


转换完毕，样本个数： 700
转换完毕，样本个数： 164
转换完毕，样本个数： 165
[  47.33381026   -9.67503911   -8.22311176   83.79172309    7.33977439
   41.09439278  -91.23730839 -117.01973066  -75.72460647   24.62504798
   -1.83490632  -38.57968573    1.31144634  -20.47615191   25.09380344
  -80.8660152   109.35541284   18.09958444   11.51868912   53.17818687
    6.40405091   43.67676827  -64.11598376   29.95224351 -153.94778579
   37.9948571    -8.74233765  -70.38884231    1.40611013  104.94119688
  -28.70774866  -86.56133669   20.73661254   55.59556261   14.07421954
   67.65926692  -62.55489458  -33.15947646    1.93600065  -55.95975275
  -80.78175315  -62.75935346   -0.79109673   49.29034337   16.34252421
   20.75557961   54.58805919   64.27934194    0.68587422  -99.07220448
    5.79061812  -31.32292832  -10.30711842   51.40635674   90.60440365
  -13.49232894   17.71760238  -11.80977659  -45.67592717  -16.10368052
    1.63594147   25.40058633   34.05247611  -56.34272638  -27.43536234
  -53.43331459   -5.1100856   -6

## 3.模型训练：多标签模型knn、svm

In [206]:
from sklearn.multiclass import OneVsRestClassifier #（一对多的模型，可以尝试OneVsOne）

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier




classifi = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=9))

# classifi = OneVsRestClassifier(LinearSVC())

# classifi = OneVsRestClassifier(DecisionTreeClassifier())


print(classifi)
# 训练

classifi.fit(train_x_vct, train_y_lable) 
# 预测cv集结果
predictions = classifi.predict(crossVali_x_vct)
# 观测结果
print(type(predictions[0]))
print(type(crossVali_y_lable[0]))

# predictions.astype(float)

print(predictions[0:2])
print(crossVali_y_lable[0:2])


OneVsRestClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform'),
          n_jobs=1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0 1 0 0 0 0]
 [1 0 0 0 0 0]]
[[0 0 1 1 0 0]
 [0 0 1 1 0 0]]


## 4.交叉验证cross validation，选择优参数

In [207]:
# 衡量模型好坏，然后回去train选择更好的参数
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(crossVali_y_lable , predictions, average='micro')
recall = recall_score(crossVali_y_lable, predictions, average='micro')
f1 = f1_score(crossVali_y_lable, predictions, average="micro")  # micro weighted macro 

print("Micro-average quality numbers")
print("Precision Score: ", precision)
print("Recall Score: ", recall)
print("f1-score: ", f1)


Micro-average quality numbers
Precision Score:  0.8717948717948718
Recall Score:  0.3655913978494624
f1-score:  0.5151515151515151


# 5.用test data 输出不同模型的准确度，给出展示。

In [208]:
# 先预测
predictions111 = classifi.predict(test_x_vct)

# 输出结果
print()
print('-----------------------分类指标的文本报告--------------------------')
from sklearn import metrics
print(metrics.classification_report(test_y_lable, predictions111))


-----------------------分类指标的文本报告--------------------------
             precision    recall  f1-score   support

          0       0.78      0.70      0.74        46
          1       0.92      0.14      0.25        84
          2       0.98      0.41      0.58       156
          3       0.85      0.66      0.74        44
          4       0.33      0.22      0.27         9
          5       0.00      0.00      0.00         6

avg / total       0.89      0.40      0.52       345



C:\Programming\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
